# LCA with Brightway2.5 - Ecoinvent

This notebook was designed to conduct LCAs using the brightway2.5 framework. I will add content in the future and make the code run more efficiently. For now, everything works within its limits and newcomers should have an easy entry to LCA using brightway.  

If you need to read up on literature, I provided some links about the tools, you'll be working with:  

1. [Python](https://www.python.org/)
2. [Brightway2.5](https://docs.brightway.dev/en/legacy/)
3. [Ecoinvent 3.11](https://ecoquery.ecoinvent.org/3.11/cutoff)
4. [Premise](https://premise.readthedocs.io/en/latest/) (coming soon, I hope)

Before you start, check the kernel on the top right. It should be "Python (env_bw25)" or whatever you named it in your virtual environment. If you are missing some functionality, check the brightway [cheat sheet](https://docs.brightway.dev/en/latest/content/cheatsheet/index.html).  

Enough blib blab, let's get started! 👾

### Step 0: Preparation

In [389]:
# Start by importing the brightway packages and some addons to calculate the results.
# There is no magic happening yet so don't get all excited.

# Brightway packages.
import bw2data as bd
import bw2io as bi
import bw2calc as bc

# Packages for calculation and stuff.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# System & authentication packages.
import os
from dotenv import load_dotenv, dotenv_values

# Ecoinvent packages.
from ecoinvent_interface import Settings, EcoinventRelease, ReleaseType

# Extra package to create UUIDs for your activities.
import hashlib

# Brightway version check, in case an error occurs.
print("bw2data version", bd.__version__)
print("bw2io version", bi.__version__)
print("bw2calc version", bc.__version__)

bw2data version (4, 0, 'dev56')
bw2io version 0.9.DEV38
bw2calc version 2.0.DEV22


### Step 1: Project setup

Brightway is organized in projects to separate the individual settings of your LCA. Project management functions are included and self-explanatory.  

Continue by setting up your project.

In [380]:
# Checking the projects never hurts and gives you an overview of all available projects.
bd.projects

Brightway2 projects manager with 2 objects:
	default
	template
Use `projects.report()` to get a report on all projects.

In [381]:
# Choose your project: activate an existing project or create and activate a new one.
bd.projects.set_current("template")
bd.projects

Brightway2 projects manager with 2 objects:
	default
	template
Use `projects.report()` to get a report on all projects.

### Step 2: Import Ecoinvent as background database

Before you can start importing the background database, you need to store your credentials. Otherwise, you won't have access to the data. For convenience, you can use an .env file where you store the user and passcode information locally on your machine.  
1. Create an .env file next to this notebook.
2. Get your Ecoinvent credentials and save them as variables in the .env like so:

> USER="YOUR_USERNAME"  
> PASSCODE="YOUR_PASSWORD"

The .env keeps your credentials out of this notebook.

In [382]:
# Check the current databases in your active project.
bd.databases

Databases dictionary with 5 object(s):
	Silicon wafer production LCA
	ecoinvent-3.10.1-biosphere
	ecoinvent-3.10.1-cutoff
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff

In [383]:
# Authenticate to Ecoinvent to check available databases.
load_dotenv()
auth = Settings(username=os.getenv('USER'), password=os.getenv('PASSCODE'))
release = EcoinventRelease(auth)
release.list_versions()

['3.11',
 '3.10.1',
 '3.10',
 '3.9.1',
 '3.9',
 '3.8',
 '3.7.1',
 '3.7',
 '3.6',
 '3.5',
 '3.4',
 '3.3',
 '3.2',
 '3.1',
 '3.01',
 '2']

In [384]:
# Insert the db version you want to work with to check for the avialable system models.
release.list_system_models('3.11')

['cutoff', 'consequential', 'apos']

In [385]:
# Specify the database version and model in variables.
db_version='3.11'
db_system_model='cutoff'

In [386]:
# Preemptive check, if the desired database is already present.
# If not: Importing database.. This may take some time.
if f'ecoinvent-{db_version}-{db_system_model}' in bd.databases:
    print(f'ecoinvent {db_version} is already present in the project')
else:
    bi.import_ecoinvent_release(
        version=db_version,
        system_model=db_system_model,
        username=os.getenv('USER'),
        password=os.getenv('PASSCODE')
    )

ecoinvent 3.11 is already present in the project


In [387]:
# Checking project databases
bd.databases

Databases dictionary with 5 object(s):
	Silicon wafer production LCA
	ecoinvent-3.10.1-biosphere
	ecoinvent-3.10.1-cutoff
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff

### Step 3: Match foreground database

Your foreground database contains the LCI data of your product system. This has to be prepared in an excel file. I included an example file with explainations and hints on how to structure your LCI data in order for the import to function correctly.  

> You can continue with the template to see how it works OR prepare your LCI data for your LCA.  

#### Creating codes for activities

As each activity has to be equipped with an unique code, I included the following hash function. You can insert an activity and use the generated hash as its code in your excel file.

In [392]:
# Optional: Hash your activities to create individual codes for them.
activity = "M12 silicon wafer production"

# Calculate the MD5 hash
code = hashlib.md5(activity.encode()).hexdigest()
print(code)

1488553895b0f8eceb7a0e7473cad56b


#### Import foreground database

At this point, you have to decide, which background database you want to use in your LCA. Per default, the declared version of the previous step will be used. If you wish to change that, adjust the variables in Step 2 accordingly.

In [393]:
# Set the foreground database.
excel_file = "lci_silicon_wafer_production.xlsx"
fg_db = bi.ExcelImporter(excel_file)

# Setting variables for background database.
bg_db = f'ecoinvent-{db_version}'
bg_sys = f'ecoinvent-{db_version}-{db_system_model}'
bg_bio = f'ecoinvent-{db_version}-biosphere'

# Matching activities against themselfs and against ecoinvent system model and biosphere to check for inconsistencies.
fg_db.apply_strategies()
fg_db.match_database(fields=["name", "unit", "reference product", "location"])
fg_db.match_database(bg_sys, fields=["name", "unit", "location", "reference product"])
fg_db.match_database(bg_bio, fields=["name", "categories", "location"])

# Checking for unlinked exchanges.
fg_db.statistics()

Extracted 3 worksheets in 0.44 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 30.21 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
3 datasets
	3 exchanges
	Links to the follo

(3, 3, 0, 0)

In [397]:
# If no unlinked exchanges are present, save the foreground database to your project.
if fg_db.db_name in bd.databases:
    del bd.databases[fg_db.db_name]
    fg_db.write_database()
else:
    fg_db.write_database()

bd.Database(fg_db.db_name).rename(f'{fg_db.db_name}_{bg_db}')

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1530.58it/s]

Vacuuming database 


Created database: Silicon wafer production LCA
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1487.87it/s]

Vacuuming database 


Brightway2 SQLiteBackend: Silicon wafer production LCA_ecoinvent-3.11

In [403]:
# Checking project databases
bd.databases

# Deleting database
# del bd.databases["<database_name>"]

Databases dictionary with 4 object(s):
	ecoinvent-3.10.1-biosphere
	ecoinvent-3.10.1-cutoff
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff

### Step 4: Calculation setup

#### Functional Unit

With our data in place, we now have to define the parameters of our LCA. In our excel file, the data is structured in processes and activities. Brightway lets us choose the number of activities (as functional units) and methods for our LCA.  

Tbd.

In [404]:
# Checking project databases
bd.databases

Databases dictionary with 4 object(s):
	ecoinvent-3.10.1-biosphere
	ecoinvent-3.10.1-cutoff
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff

In [265]:
# Choose the database for the LCA
wb = bd.Database("<database_name>")

# User choice: calculate all activities or specific ones
user_choice = input("Do you want to calculate all activities or specific ones? (type 'all' or 'specific'): ").strip().lower()

if user_choice == 'all':
    # If the user wants all activities, loop through them
    fu = {act["code"]: {wb.get(act["code"]).id: 1} for act in wb}
elif user_choice == 'specific':
    # Print all available activities
    print("Available Activities:")
    for act in wb:
        print(act['name'])
        
    # Ask the user to enter the names of the activities they want
    specific_names = input("Enter the names of the activities you want to include, separated by commas: ").split(',')

    # Create the functional unit based on user input
    fu = {}
    for name in specific_names:
        cleaned_name = name.strip()  # Remove any extra spaces
        # Find the corresponding activity by name
        matching_activities = [act for act in wb if act['name'].strip().lower() == cleaned_name.lower()]
        
        if matching_activities:
            for act in matching_activities:
                fu[act["code"]] = {wb.get(act["code"]).id: 1}
        else:
            print(f"Warning: Activity with name '{cleaned_name}' not found in database.")
else:
    print("Invalid choice. Please type 'all' or 'specific'.")

print("The functional unit has been set accordingly.")

Do you want to calculate all activities or specific ones? (type 'all' or 'specific'):  specific


Available Activities:
M12 silicon wafer production
M12 silicon wafer cleaning
M12 silicon ingot production


Enter the names of the activities you want to include, separated by commas:  M12 silicon wafer production, M12 silicon ingot production


The functional unit has been set accordingly.


#### LCIA method

Brightway allowes most LCIA methods to be used in the LCA. Available LCIA methods depend on the database version used in the background.  
> Note: Each LCIA method contains various characterization factors or indicators.  
> E.g. climate change, acidification, water use, etc.

For customization purposes, this template lists all available methods from which you can choose.

In [294]:
# Use a set to store unique second entries
lcia_methods = set()

# Iterate through the data structure and add the second element of each tuple to the set
for method in list(filter(lambda x: bg_db in x[0], bd.methods)):
    lcia_methods.add(method[1])

print(sorted(lcia_methods))

['CML v4.8 2016', 'CML v4.8 2016 no LT', 'Crustal Scarcity Indicator 2020', 'Cumulative Energy Demand (CED)', 'Cumulative Exergy Demand (CExD)', 'EF v3.0', 'EF v3.0 no LT', 'EF v3.1', 'EF v3.1 no LT', 'EN15804+A2 - Additional impact categories and indicators', 'EN15804+A2 - Core impact categories and indicators', 'EN15804+A2 - Indicators describing output flows', 'EN15804+A2 - Indicators describing resource use', 'EN15804+A2 - Indicators describing waste categories', 'EPS 2020d', 'EPS 2020d no LT', 'Ecological Footprint', 'Ecological Scarcity 2021', 'Ecological Scarcity 2021 no LT', 'Ecosystem Damage Potential', 'IMPACT World+ v2.1, footprint version', 'IPCC 2013', 'IPCC 2013 no LT', 'IPCC 2021', 'IPCC 2021 (incl. biogenic CO2)', 'IPCC 2021 (incl. biogenic CO2) no LT', 'IPCC 2021 no LT', 'Inventory results and indicators', 'ReCiPe 2016 v1.03, endpoint (E)', 'ReCiPe 2016 v1.03, endpoint (E) no LT', 'ReCiPe 2016 v1.03, endpoint (H)', 'ReCiPe 2016 v1.03, endpoint (H) no LT', 'ReCiPe 2016 

In [351]:
# Choose your LCIA method
lcia_method = "<method_from_list>"

In [352]:
# Setting the method config and LCA object.
my_methods = list(filter(lambda x: lcia_method in x[1] and x[0].startswith(bg_db), bd.methods))
print("We have", len(my_methods), f"{lcia_method} methods we will evaluate.")

config = {
    "impact_categories": my_methods
}

# Create our final LCA object used for the calculation in the next step.
data_objs = bd.get_multilca_data_objs(functional_units=fu, method_config=config)

We have 18 ReCiPe 2016 v1.03, midpoint (H) no LT methods we will evaluate.


### Step 5: Calculation

In [353]:
# TODO: Ask Chris, why bc.LCA is used. Isn't it possible, to use bc.MultiLCA for single LCA calcs?
# lca = bc.LCA({our_activity: 1}, ef_gwp_key)

# Calculate MultiLCA
lca = bc.MultiLCA(demands=fu, method_config=config, data_objs=data_objs)
lca.lci()
lca.lcia()

# Choose between single or multi LCA
# lca.score
# Format the results
raw_results = lca.scores
def transform_raw_results(raw_results, wb):
    transformed_results = {}
    
    for (key, hash_value), outcome in raw_results.items():
        # Extract the third value from the first part of the key
        category = key[2]  # This corresponds to the third value (e.g., 'climate change', 'human health', etc.)
        
        # Use wb.get() to fetch the activity name directly
        activity_name = wb.get(hash_value)["name"]
        
        # Construct the new key
        new_key = (activity_name, category)
        
        # Add the new key and outcome to the transformed results
        transformed_results[new_key] = outcome
    
    return transformed_results

lca_results = transform_raw_results(raw_results, wb)

In [354]:
# Display the results in a table
dfresults = pd.DataFrame.from_dict(lca_results, orient='index')
dfresults.index = pd.MultiIndex.from_tuples(dfresults.index, names=('Activities', lcia_method))
dfresults = dfresults.unstack(level=lcia_method)
dfresults.columns = dfresults.columns.droplevel(0)
dfresults

"ReCiPe 2016 v1.03, midpoint (H) no LT",acidification: terrestrial no LT,climate change no LT,ecotoxicity: freshwater no LT,ecotoxicity: marine no LT,ecotoxicity: terrestrial no LT,"energy resources: non-renewable, fossil no LT",eutrophication: freshwater no LT,eutrophication: marine no LT,human toxicity: carcinogenic no LT,human toxicity: non-carcinogenic no LT,ionising radiation no LT,land use no LT,material resources: metals/minerals no LT,ozone depletion no LT,particulate matter formation no LT,photochemical oxidant formation: human health no LT,photochemical oxidant formation: terrestrial ecosystems no LT,water use no LT
Activities,,,,,,,,,,,,,,,,,,
M12 silicon ingot production,0.105870,39.905342,0.008977,0.034954,26.323168,11.387785,0.002009,0.000522,0.093474,2.756869,0.700177,1.382015,0.548782,1.318863e-05,0.046754,0.070456,0.073270,0.373432
M12 silicon wafer production,0.001059,0.399053,0.000090,0.000350,0.263232,0.113878,0.000020,0.000005,0.000935,0.027569,0.007002,0.013820,0.005488,1.318863e-07,0.000468,0.000705,0.000733,0.003734
